In [1]:
import torch
import torch.nn as nn
import numpy as np
import time
import math
from matplotlib import pyplot

# Class

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]

In [3]:
class TransAm(nn.Module):
    def __init__(self,feature_size=250,num_layers=1,dropout=0.1):
        super(TransAm, self).__init__()
        self.model_type = 'Transformer'
        
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src): # 예제랑 다른부분: embedding이 없으므로 encoder에 넣지 않음
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)#, self.src_mask)
        output = self.decoder(output)
        return output

    def _generate_square_subsequent_mask(self, sz): # 예제랑 다른부분: 더 복잡함, float을 만들기 위함인가?
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


In [4]:
# if window is 100 and prediction step is 1
# in -> [0..99]
# target -> [1..100]
def create_inout_sequences(input_data, tw): # Size:[input_data-tw, 2, tw] Ex(2500, 2, 100)
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = np.append(input_data[i:i+tw][:-output_window], output_window*[0]) # 100개중 95개는 input_data값, 마지막 5개는 0
        train_label = input_data[i:i+tw]
        inout_seq.append((train_seq ,train_label))
    return torch.FloatTensor(inout_seq)

In [5]:
timestep        = np.arange(0, 400, 0.1)    
amplitude   = np.sin(timestep) + np.sin(timestep*0.05) +np.sin(timestep*0.12) *np.random.normal(-0.2, 0.2, len(timestep))


from sklearn.preprocessing import MinMaxScaler

#loading weather data from a file
#from pandas import read_csv
#series = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)

# looks like normalizing input values curtial for the model
scaler = MinMaxScaler(feature_range=(-1, 1)) 
#amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)


sampels = 2600
train_data = amplitude[:sampels]
test_data = amplitude[sampels:]

In [6]:
def get_data():
    # construct a littel toy dataset
    time        = np.arange(0, 400, 0.1)    
    amplitude   = np.sin(time) + np.sin(time*0.05) +np.sin(time*0.12) *np.random.normal(-0.2, 0.2, len(time))

    
    from sklearn.preprocessing import MinMaxScaler
    
    #loading weather data from a file
    #from pandas import read_csv
    #series = read_csv('daily-min-temperatures.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    
    # looks like normalizing input values curtial for the model
    scaler = MinMaxScaler(feature_range=(-1, 1)) 
    #amplitude = scaler.fit_transform(series.to_numpy().reshape(-1, 1)).reshape(-1)
    amplitude = scaler.fit_transform(amplitude.reshape(-1, 1)).reshape(-1)
    
    
    sampels = 2800
    train_data = amplitude[:sampels]
    test_data = amplitude[sampels:]

    # convert our train data into a pytorch train tensor
    #train_tensor = torch.FloatTensor(train_data).view(-1)
    # todo: add comment.. 
    train_sequence = create_inout_sequences(train_data,input_window)
    train_sequence = train_sequence[:-output_window] #todo: fix hack? -> din't think this through, looks like the last n sequences are to short, so I just remove them. Hackety Hack.. 

    #test_data = torch.FloatTensor(test_data).view(-1) 
    test_data = create_inout_sequences(test_data,input_window)
    test_data = test_data[:-output_window] #todo: fix hack?

    return train_sequence.to(device),test_data.to(device)

In [7]:
def get_batch(source, i,batch_size):
    seq_len = min(batch_size, len(source) - 1 - i)
    data = source[i:i+seq_len]    
    input = torch.stack(torch.stack([item[0] for item in data]).chunk(input_window,1)) # 1 is feature size
    target = torch.stack(torch.stack([item[1] for item in data]).chunk(input_window,1))
    return input, target

In [8]:
def train(train_data):
    model.train() # Turn on the train mode
    total_loss = 0.
    start_time = time.time()

    for batch, i in enumerate(range(0, len(train_data) - 1, batch_size)):
        data, targets = get_batch(train_data, i,batch_size)
        optimizer.zero_grad()
        output = model(data)        

        if calculate_loss_over_all_values:
            loss = criterion(output, targets)
        else:
            loss = criterion(output[-output_window:], targets[-output_window:])
    
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        log_interval = int(len(train_data) / batch_size / 5)
        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | '
                  'lr {:02.6f} | {:5.2f} ms | '
                  'loss {:5.5f} | ppl {:8.2f}'.format(
                    epoch, batch, len(train_data) // batch_size, scheduler.get_last_lr()[0],
                    elapsed * 1000 / log_interval,
                    cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()

In [9]:
def plot_and_loss(eval_model, data_source,epoch):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    with torch.no_grad():
        for i in range(0, len(data_source) - 1):
            data, target = get_batch(data_source, i,1)
            output = eval_model(data)                        
            if calculate_loss_over_all_values:                                
                total_loss += criterion(output, target).item()
            else:
                total_loss += criterion(output[-output_window:], target[-output_window:]).item()
            
            test_result = torch.cat((test_result, output[-1].view(-1).cpu()), 0) # tensor를 numpy로 변환하여 활용하는 기법인것같군
            truth = torch.cat((truth, target[-1].view(-1).cpu()), 0)
            
    #test_result = test_result.cpu().numpy() -> no need to detach stuff.. 
    len(test_result)

    pyplot.plot(test_result,color="red")
    pyplot.plot(truth[:500],color="blue")
    pyplot.plot(test_result-truth,color="green")
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer-epoch_multi%d.png'%epoch)
    pyplot.close()
    
    return total_loss / i

In [10]:
# predict the next n steps based on the input data 
def predict_future(eval_model, data_source,steps):
    eval_model.eval() 
    total_loss = 0.
    test_result = torch.Tensor(0)    
    truth = torch.Tensor(0)
    data, _ = get_batch(data_source, 0,1)
    with torch.no_grad():
        for i in range(0, steps):            
            output = eval_model(data[-input_window:])                        
            data = torch.cat((data, output[-1:]))
            
    data = data.cpu().view(-1)
    
    # I used this plot to visualize if the model pics up any long therm struccture within the data. 
    pyplot.plot(data,color="red")       
    pyplot.plot(data[:input_window],color="blue")    
    pyplot.grid(True, which='both')
    pyplot.axhline(y=0, color='k')
    pyplot.savefig('graph/transformer_multi-future%d.png'%steps)
    pyplot.close()
        

In [11]:
def evaluate(eval_model, data_source):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.
    eval_batch_size = 1000
    with torch.no_grad():
        for i in range(0, len(data_source) - 1, eval_batch_size):
            data, targets = get_batch(data_source, i,eval_batch_size)
            output = eval_model(data)            
            if calculate_loss_over_all_values:
                total_loss += len(data[0])* criterion(output, targets).cpu().item()
            else:                                
                total_loss += len(data[0])* criterion(output[-output_window:], targets[-output_window:]).cpu().item()            
    return total_loss / len(data_source)

# Parameters

In [12]:
torch.manual_seed(0)
np.random.seed(0)

In [13]:
calculate_loss_over_all_values = False 

In [14]:
input_window = 100 # number of input steps
output_window = 5 
batch_size = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
train_data, val_data = get_data()
model = TransAm().to(device)

In [16]:
criterion = nn.MSELoss()
lr = 0.005 
#optimizer = torch.optim.SGD(model.parameters(), lr=lr)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.98)

best_val_loss = float("inf")
epochs = 100 # The number of epochs
best_model = None

# Main

In [17]:
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    train(train_data)


    if(epoch % 10 is 0):
        val_loss = plot_and_loss(model, val_data,epoch)
        predict_future(model, val_data,200)
    else:
        val_loss = evaluate(model, val_data)

    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.5f} | valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    #if val_loss < best_val_loss:
    #    best_val_loss = val_loss
    #    best_model = model

    scheduler.step() 


| epoch   1 |    53/  269 batches | lr 0.005000 | 20.44 ms | loss 5.45161 | ppl   233.13
| epoch   1 |   106/  269 batches | lr 0.005000 | 11.07 ms | loss 0.17886 | ppl     1.20
| epoch   1 |   159/  269 batches | lr 0.005000 | 11.01 ms | loss 0.09819 | ppl     1.10
| epoch   1 |   212/  269 batches | lr 0.005000 | 11.12 ms | loss 0.15666 | ppl     1.17
| epoch   1 |   265/  269 batches | lr 0.005000 | 11.39 ms | loss 0.04705 | ppl     1.05
-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  3.60s | valid loss 0.16842 | valid ppl     1.18
-----------------------------------------------------------------------------------------
| epoch   2 |    53/  269 batches | lr 0.004900 | 11.10 ms | loss 0.07835 | ppl     1.08
| epoch   2 |   106/  269 batches | lr 0.004900 | 10.43 ms | loss 0.10857 | ppl     1.11
| epoch   2 |   159/  269 batches | lr 0.004900 | 10.44 ms | loss 0.07377 | ppl     1.08
| epoch   2 |   212/  269 batche

| epoch  13 |    53/  269 batches | lr 0.003924 | 10.43 ms | loss 0.02601 | ppl     1.03
| epoch  13 |   106/  269 batches | lr 0.003924 | 10.15 ms | loss 0.01011 | ppl     1.01
| epoch  13 |   159/  269 batches | lr 0.003924 |  9.99 ms | loss 0.01015 | ppl     1.01
| epoch  13 |   212/  269 batches | lr 0.003924 | 10.08 ms | loss 0.01313 | ppl     1.01
| epoch  13 |   265/  269 batches | lr 0.003924 | 10.13 ms | loss 0.01004 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  13 | time:  2.81s | valid loss 0.26613 | valid ppl     1.30
-----------------------------------------------------------------------------------------
| epoch  14 |    53/  269 batches | lr 0.003845 | 10.35 ms | loss 0.02497 | ppl     1.03
| epoch  14 |   106/  269 batches | lr 0.003845 | 10.34 ms | loss 0.00981 | ppl     1.01
| epoch  14 |   159/  269 batches | lr 0.003845 | 10.01 ms | loss 0.01058 | ppl     1.01
| epoch  14 |   212/  269 batche

| epoch  25 |    53/  269 batches | lr 0.003079 | 10.35 ms | loss 0.03604 | ppl     1.04
| epoch  25 |   106/  269 batches | lr 0.003079 | 10.36 ms | loss 0.01437 | ppl     1.01
| epoch  25 |   159/  269 batches | lr 0.003079 | 10.34 ms | loss 0.01138 | ppl     1.01
| epoch  25 |   212/  269 batches | lr 0.003079 | 11.17 ms | loss 0.00955 | ppl     1.01
| epoch  25 |   265/  269 batches | lr 0.003079 | 10.27 ms | loss 0.01268 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  25 | time:  2.87s | valid loss 0.25438 | valid ppl     1.29
-----------------------------------------------------------------------------------------
| epoch  26 |    53/  269 batches | lr 0.003017 | 10.46 ms | loss 0.03322 | ppl     1.03
| epoch  26 |   106/  269 batches | lr 0.003017 |  9.76 ms | loss 0.01521 | ppl     1.02
| epoch  26 |   159/  269 batches | lr 0.003017 | 11.11 ms | loss 0.01415 | ppl     1.01
| epoch  26 |   212/  269 batche

| epoch  37 |    53/  269 batches | lr 0.002416 | 10.53 ms | loss 0.02792 | ppl     1.03
| epoch  37 |   106/  269 batches | lr 0.002416 | 10.58 ms | loss 0.03523 | ppl     1.04
| epoch  37 |   159/  269 batches | lr 0.002416 | 10.44 ms | loss 0.02008 | ppl     1.02
| epoch  37 |   212/  269 batches | lr 0.002416 | 10.33 ms | loss 0.01280 | ppl     1.01
| epoch  37 |   265/  269 batches | lr 0.002416 | 11.24 ms | loss 0.01046 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  37 | time:  2.93s | valid loss 0.18107 | valid ppl     1.20
-----------------------------------------------------------------------------------------
| epoch  38 |    53/  269 batches | lr 0.002368 | 10.95 ms | loss 0.02672 | ppl     1.03
| epoch  38 |   106/  269 batches | lr 0.002368 | 11.53 ms | loss 0.03679 | ppl     1.04
| epoch  38 |   159/  269 batches | lr 0.002368 | 10.57 ms | loss 0.02642 | ppl     1.03
| epoch  38 |   212/  269 batche

| epoch  49 |    53/  269 batches | lr 0.001896 | 10.36 ms | loss 0.01693 | ppl     1.02
| epoch  49 |   106/  269 batches | lr 0.001896 | 10.65 ms | loss 0.01784 | ppl     1.02
| epoch  49 |   159/  269 batches | lr 0.001896 | 10.66 ms | loss 0.01915 | ppl     1.02
| epoch  49 |   212/  269 batches | lr 0.001896 | 10.80 ms | loss 0.01523 | ppl     1.02
| epoch  49 |   265/  269 batches | lr 0.001896 |  9.91 ms | loss 0.01507 | ppl     1.02
-----------------------------------------------------------------------------------------
| end of epoch  49 | time:  2.87s | valid loss 0.05893 | valid ppl     1.06
-----------------------------------------------------------------------------------------
| epoch  50 |    53/  269 batches | lr 0.001858 | 10.53 ms | loss 0.01697 | ppl     1.02
| epoch  50 |   106/  269 batches | lr 0.001858 | 10.37 ms | loss 0.01923 | ppl     1.02
| epoch  50 |   159/  269 batches | lr 0.001858 | 10.13 ms | loss 0.01659 | ppl     1.02
| epoch  50 |   212/  269 batche

| epoch  61 |    53/  269 batches | lr 0.001488 | 10.41 ms | loss 0.01460 | ppl     1.01
| epoch  61 |   106/  269 batches | lr 0.001488 | 10.10 ms | loss 0.01042 | ppl     1.01
| epoch  61 |   159/  269 batches | lr 0.001488 | 10.03 ms | loss 0.01294 | ppl     1.01
| epoch  61 |   212/  269 batches | lr 0.001488 | 10.69 ms | loss 0.01071 | ppl     1.01
| epoch  61 |   265/  269 batches | lr 0.001488 | 11.83 ms | loss 0.01173 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  61 | time:  2.92s | valid loss 0.01982 | valid ppl     1.02
-----------------------------------------------------------------------------------------
| epoch  62 |    53/  269 batches | lr 0.001458 | 12.13 ms | loss 0.01330 | ppl     1.01
| epoch  62 |   106/  269 batches | lr 0.001458 | 11.54 ms | loss 0.01162 | ppl     1.01
| epoch  62 |   159/  269 batches | lr 0.001458 | 10.75 ms | loss 0.01268 | ppl     1.01
| epoch  62 |   212/  269 batche

| epoch  73 |    53/  269 batches | lr 0.001167 | 10.35 ms | loss 0.01195 | ppl     1.01
| epoch  73 |   106/  269 batches | lr 0.001167 | 10.06 ms | loss 0.00976 | ppl     1.01
| epoch  73 |   159/  269 batches | lr 0.001167 | 10.06 ms | loss 0.01117 | ppl     1.01
| epoch  73 |   212/  269 batches | lr 0.001167 | 10.35 ms | loss 0.01011 | ppl     1.01
| epoch  73 |   265/  269 batches | lr 0.001167 | 10.15 ms | loss 0.01035 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  73 | time:  2.81s | valid loss 0.01663 | valid ppl     1.02
-----------------------------------------------------------------------------------------
| epoch  74 |    53/  269 batches | lr 0.001144 | 10.36 ms | loss 0.01152 | ppl     1.01
| epoch  74 |   106/  269 batches | lr 0.001144 | 11.07 ms | loss 0.01010 | ppl     1.01
| epoch  74 |   159/  269 batches | lr 0.001144 | 10.22 ms | loss 0.01056 | ppl     1.01
| epoch  74 |   212/  269 batche

| epoch  85 |    53/  269 batches | lr 0.000916 | 10.38 ms | loss 0.01002 | ppl     1.01
| epoch  85 |   106/  269 batches | lr 0.000916 | 10.05 ms | loss 0.00869 | ppl     1.01
| epoch  85 |   159/  269 batches | lr 0.000916 | 10.23 ms | loss 0.01022 | ppl     1.01
| epoch  85 |   212/  269 batches | lr 0.000916 | 10.60 ms | loss 0.00928 | ppl     1.01
| epoch  85 |   265/  269 batches | lr 0.000916 | 10.59 ms | loss 0.00926 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  85 | time:  2.85s | valid loss 0.01608 | valid ppl     1.02
-----------------------------------------------------------------------------------------
| epoch  86 |    53/  269 batches | lr 0.000898 | 10.78 ms | loss 0.00933 | ppl     1.01
| epoch  86 |   106/  269 batches | lr 0.000898 | 10.86 ms | loss 0.00902 | ppl     1.01
| epoch  86 |   159/  269 batches | lr 0.000898 | 10.17 ms | loss 0.00966 | ppl     1.01
| epoch  86 |   212/  269 batche

| epoch  97 |    53/  269 batches | lr 0.000719 | 10.57 ms | loss 0.00839 | ppl     1.01
| epoch  97 |   106/  269 batches | lr 0.000719 |  9.98 ms | loss 0.00794 | ppl     1.01
| epoch  97 |   159/  269 batches | lr 0.000719 | 10.33 ms | loss 0.00811 | ppl     1.01
| epoch  97 |   212/  269 batches | lr 0.000719 | 10.35 ms | loss 0.00833 | ppl     1.01
| epoch  97 |   265/  269 batches | lr 0.000719 | 10.15 ms | loss 0.00846 | ppl     1.01
-----------------------------------------------------------------------------------------
| end of epoch  97 | time:  2.82s | valid loss 0.01477 | valid ppl     1.01
-----------------------------------------------------------------------------------------
| epoch  98 |    53/  269 batches | lr 0.000705 | 10.83 ms | loss 0.00795 | ppl     1.01
| epoch  98 |   106/  269 batches | lr 0.000705 | 10.35 ms | loss 0.00806 | ppl     1.01
| epoch  98 |   159/  269 batches | lr 0.000705 | 10.07 ms | loss 0.00832 | ppl     1.01
| epoch  98 |   212/  269 batche